In [ ]:
#@title Import Libraries
# Importing Libraries
import numpy as np
import math
from numba import jit, cuda
import time

In [ ]:
#@title Data

# Test the modified elliptical_gaussian function
x = np.linspace(-5, 5, 100)
y = np.linspace(-5, 5, 100)
amp = 1.0
xo = 0.0
yo = 0.0
sx = 1.0
sy = 0.5
theta = 45.0

In [ ]:
#@title Pythonic Implementation
# Running the function on Python
# Original Function
def elliptical_gaussian(x, y, amp, xo, yo, sx, sy, theta):
    """
    Generate a model 2d Gaussian with the given parameters.
    Evaluate this model at the given locations x,y.

    Parameters
    ----------
    x, y : numeric or array-like
        locations at which to evaluate the gaussian
    amp : float
        Peak value.
    xo, yo : float
        Center of the gaussian.
    sx, sy : float
        major/minor axes in sigmas
    theta : float
        position angle (degrees) CCW from x-axis

    Returns
    -------
    data : numeric or array-like
        Gaussian function evaluated at the x,y locations.
    """
    try:
        sint, cost = math.sin(np.radians(theta)), math.cos(np.radians(theta))
    except ValueError as e:
        if 'math domain error' in e.args:
            sint, cost = np.nan, np.nan
    xxo = x - xo
    yyo = y - yo
    exp = (xxo * cost + yyo * sint) ** 2 / sx ** 2 \
        + (xxo * sint - yyo * cost) ** 2 / sy ** 2
    exp *= -1. / 2
    return amp * np.exp(exp)

total_time = 0
num_iterations = 10

for _ in range(num_iterations):
    start_time = time.time()
    for i in range(1_000_000):
        result = elliptical_gaussian(x, y, amp, xo, yo, sx, sy, theta)
    end_time = time.time()
    execution_time2 = end_time - start_time
    total_time += execution_time2

average_execution_time_python = total_time / num_iterations
print(f"Average execution time: {average_execution_time_python} seconds")


Average execution time: 23.702743315696715 seconds


In [ ]:
#@title Numba CPU
# Running the function on Numba
from numba import jit, prange
import numpy as np
import math
import time

# Running the function on Numba
@jit(nopython=True, parallel=True)
def elliptical_gaussian(x, y, amp, xo, yo, sx, sy, theta):
    result = np.zeros_like(x)
    for i in prange(x.shape[0]):
        if theta % 90 == 0:
            sint, cost = 0.0, 1.0
        else:
            sint, cost = math.sin(np.radians(theta)), math.cos(np.radians(theta))
        xxo = x[i] - xo
        yyo = y[i] - yo
        exp = ((xxo * cost + yyo * sint) ** 2 / sx ** 2 + (xxo * sint - yyo * cost) ** 2 / sy ** 2) / -2
        result[i] = amp * math.exp(exp)
    return result

# Test parameters
x = np.linspace(-5, 5, 100)
y = np.linspace(-5, 5, 100)
amp = 1.0
xo = 0.0
yo = 0.0
sx = 1.0
sy = 0.5
theta = 45.0

total_time = 0
num_iterations = 10

for _ in range(num_iterations):
    start_time = time.time()
    for i in range(1_000_000):
        result = elliptical_gaussian(x, y, amp, xo, yo, sx, sy, theta)
    end_time = time.time()
    execution_time2 = end_time - start_time
    total_time += execution_time2

average_execution_time_numba_cpu = total_time / num_iterations
print(f"Average execution time: {average_execution_time_numba_cpu} seconds")


Average execution time: 12.478839921951295 seconds


In [ ]:
#@title Percentage Comparison between Python and Numba
# Percentage comparison between the Python and Numba implementations
percentage = (average_execution_time_python - average_execution_time_numba_cpu) / average_execution_time_python * 100
print(f"Percentage improvement between Python and Numba: {percentage}%")

Percentage improvement between Python and Numba: 47.35276100430363%


In [ ]:
#@title Numba GPU
from numba import cuda, float64, int64

@cuda.jit
def elliptical_gaussian_gpu(x, y, result, amp, xo, yo, sx, sy, theta):
    i = cuda.grid(1)
    if i < x.shape[0]:
        if theta % 90 == 0:
            sint, cost = 0.0, 1.0
        else:
            sint, cost = math.sin(np.radians(theta)), math.cos(np.radians(theta))
        xxo = x[i] - xo
        yyo = y[i] - yo
        exp = ((xxo * cost + yyo * sint) ** 2 / sx ** 2 + (xxo * sint - yyo * cost) ** 2 / sy ** 2) / -2
        result[i] = amp * math.exp(exp)

# Test the modified elliptical_gaussian function
x = np.linspace(-5, 5, 100)
y = np.linspace(-5, 5, 100)
amp = 1.0
xo = 0.0
yo = 0.0
sx = 1.0
sy = 0.5
theta = 45.0

# Allocate memory on the GPU
d_x = cuda.to_device(x)
d_y = cuda.to_device(y)
d_result = cuda.device_array_like(x)

# Define the block size and grid size
block_size = 256
grid_size = (x.shape[0] + block_size - 1) // block_size

start_time = time.time()
# Profile the elliptical_gaussian_gpu function for each set of dummy data
for i in range(1_000_000):
    elliptical_gaussian_gpu[grid_size, block_size](d_x, d_y, d_result, amp, xo, yo, sx, sy, theta)

end_time = time.time()
average_execution_time_gpu = end_time - start_time
print(f"Total execution time: {average_execution_time_gpu} seconds")


/usr/local/lib/python3.10/dist-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


Total execution time: 77.7383406162262 seconds


In [ ]:
#@title Percentage Comparison between Python and Numba GPU
# Percentage comparison between the Python and Numba implementations
percentage = (average_execution_time_python - average_execution_time_gpu) / average_execution_time_python * 100
print(f"Percentage improvement between Python and Numba: {percentage}%")

Percentage improvement between Python and Numba: -227.97191270575578%
